In [1]:
import pormake as pm
import numpy as np
import csv
from collections import defaultdict

db = pm.Database()
loc = pm.Locator()


2023-07-22 18:19:39.185679: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-22 18:19:39.221576: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-22 18:19:39.222474: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-22 18:19:39.830713: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def expand_topo(topo):
    x = 1
    y = 1
    z = 1
    edges = topo.edge_indices
    neigh = topo.neighbor_list
    for i in edges:

        ## below set lists unique neighbors for the edges. If 1, this means expansion is required.
        if len(set([j.index for j in neigh[i]])) == 1:
            dist = [abs(d) for d in neigh[i][0].distance_vector]

            ## expand topology along the "major" edge direction to allow for alternating node sequence.
            if x == 1 and dist.index(max(dist)) == 0:
                x = 2
            if y == 1 and dist.index(max(dist)) == 1:
                y = 2
            if z == 1 and dist.index(max(dist)) == 2:
                z = 2

    return topo * (x, y, z)

In [3]:
## prep topologies for patterned node assignment

topo_codes = []
file = open('cn34_pid_topology_list.csv')
csvreader = csv.reader(file)

for row in csvreader:
    for topo in row:
        topo_codes.append(topo)

topos = []

for topo in topo_codes:
    cur_topo = db.get_topo(topo)
    cur_topo = expand_topo(cur_topo)

    if cur_topo.n_nodes % 2 == 1:
        ## expand in one of the directions to ensure even number of nodes
        cur_topo = cur_topo * (1, 1, 2)
        
    topos.append(cur_topo)

In [4]:
## check rmsd and topology scale here (check # nodes again since expansion took place)

node_limit = 80
refined_topos = []

for topo in topos:
    
    if topo.n_nodes > node_limit:
        continue
    refined_topos.append(topo)


In [5]:
## attempt generation!
test_builder = pm.Builder(planarity_enforcement=True, angle_threshold=10, check_tetrahedral=True)

TP_O = pm.BuildingBlock(bb_file="pi-d_building_blocks/TP_O.xyz")
PcM_O = pm.BuildingBlock(bb_file="pi-d_building_blocks/PcM_O.xyz")
Fe_pln = pm.BuildingBlock(bb_file="pi-d_building_blocks/Fe_pln.xyz")

successful_cases = []

for topo in refined_topos:
    
    print(topo.name)
        
    current_node = {}
    current_edge = {}    
    
    ## initialize nodes with Fe_oct first (no chirality dependence... is this good?)
    for i, node in enumerate(topo.unique_cn):
        
        if node == 3:
            current_node[i] = TP_O
        elif node == 4:
            current_node[i] = PcM_O
        else:
            print('something wrong with', topo.name)

    
    ## initialize edges
    for i, edge in enumerate(topo.unique_edge_types):
        current_edge[tuple(edge)] = Fe_pln

    bbs = test_builder.make_bbs_by_type(topo, current_node, current_edge)
    calc_permutation = test_builder.extract_permutation(topo, current_node, current_edge)
    calc_permutation = {i : perm for i, perm in enumerate(calc_permutation)}
    
    try:
        mof = test_builder.build(topo, bbs, calc_permutation)
        mof.view()
        successful_cases.append(topo.name)

    except:
        continue


>>> There are atoms without bond: TP_O, [(33, 'Xx'), (34, 'Xx'), (35, 'Xx')].
>>> There are atoms without bond: PcM_O, [(64, 'Xx'), (56, 'Ni'), (61, 'Xx'), (62, 'Xx'), (63, 'Xx')].
>>> There are atoms without bond: Fe_pln, [(3, 'Xx'), (4, 'Xx')].
>>> == Min RMSD of (node type: 0, node bb: TP_O): 6.02E-02
>>> Pre-location at node slot 0, (node type: 0, node bb: TP_O), RMSD: 6.02E-02
>>> Pre-location at node slot 1, (node type: 0, node bb: TP_O), RMSD: 6.02E-02
>>> Pre-location at node slot 2, (node type: 0, node bb: TP_O), RMSD: 6.02E-02
>>> Pre-location at node slot 3, (node type: 0, node bb: TP_O), RMSD: 6.02E-02
>>> Pre-location at node slot 4, (node type: 0, node bb: TP_O), RMSD: 6.02E-02
>>> Pre-location at node slot 5, (node type: 0, node bb: TP_O), RMSD: 6.02E-02
>>> Pre-location at node slot 6, (node type: 0, node bb: TP_O), RMSD: 6.02E-02
>>> Pre-location at node slot 7, (node type: 0, node bb: TP_O), RMSD: 6.02E-02
>>> Pre-location at node slot 8, (node type: 0, node bb: TP_O)

bcu-f


2023-07-22 18:20:02.187009: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
>>> Topology optimization starts.
>>> MESSAGE: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
>>> SUCCESS: True
>>> ITER: 23
>>> OBJ: 0.007
>>> Location at node slot 0, (node type: 0, node bb: TP_O), RMSD: 1.14E-02
>>> Location at node slot 1, (node type: 0, node bb: TP_O), RMSD: 1.14E-02
>>> Location at node slot 2, (node type: 0, node bb: TP_O), RMSD: 1.14E-02
>>> Location at node slot 3, (node type: 0, node bb: TP_O), RMSD: 1.14E-02
>>> Location at node slot 4, (node type: 0, node bb: TP_O), RMSD: 1.14E-02
>>> Location at node slot 5, (node type: 0, node bb: TP_O), RMSD: 1.14E-02
>>> Location at

bcu-h


>>> MESSAGE: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
>>> SUCCESS: True
>>> ITER: 41
>>> OBJ: 0.016
>>> Location at node slot 0, (node type: 0, node bb: TP_O), RMSD: 3.14E-02
>>> Location at node slot 1, (node type: 0, node bb: TP_O), RMSD: 2.68E-02
>>> Location at node slot 2, (node type: 0, node bb: TP_O), RMSD: 3.10E-02
>>> Location at node slot 3, (node type: 0, node bb: TP_O), RMSD: 3.10E-02
>>> Location at node slot 4, (node type: 0, node bb: TP_O), RMSD: 3.10E-02
>>> Location at node slot 5, (node type: 0, node bb: TP_O), RMSD: 3.10E-02
>>> Location at node slot 6, (node type: 0, node bb: TP_O), RMSD: 2.68E-02
>>> Location at node slot 7, (node type: 0, node bb: TP_O), RMSD: 3.14E-02
>>> Location at node slot 8, (node type: 0, node bb: TP_O), RMSD: 3.14E-02
>>> Location at node slot 9, (node type: 0, node bb: TP_O), RMSD: 2.68E-02
>>> Location at node slot 10, (node type: 0, node bb: TP_O), RMSD: 3.10E-02
>>> Location at node slot 11, (node type: 0, node bb: TP_O), RMSD: 

...

>>> MESSAGE: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
>>> SUCCESS: True
>>> ITER: 80
>>> OBJ: 0.051
>>> Location at node slot 0, (node type: 0, node bb: TP_O), RMSD: 5.98E-02
>>> Location at node slot 1, (node type: 0, node bb: TP_O), RMSD: 6.00E-02
>>> Location at node slot 2, (node type: 0, node bb: TP_O), RMSD: 5.94E-02
>>> Location at node slot 3, (node type: 0, node bb: TP_O), RMSD: 5.98E-02
>>> Location at node slot 4, (node type: 0, node bb: TP_O), RMSD: 6.01E-02
>>> Location at node slot 5, (node type: 0, node bb: TP_O), RMSD: 5.94E-02
>>> Location at node slot 6, (node type: 0, node bb: TP_O), RMSD: 5.98E-02
>>> Location at node slot 7, (node type: 0, node bb: TP_O), RMSD: 5.94E-02
>>> Location at node slot 8, (node type: 0, node bb: TP_O), RMSD: 6.01E-02
>>> Location at node slot 9, (node type: 0, node bb: TP_O), RMSD: 5.98E-02
>>> Location at node slot 10, (node type: 0, node bb: TP_O), RMSD: 5.94E-02
>>> Location at node slot 11, (node type: 0, node bb: TP_O), RMSD: 

zme


>>> Use given permutation for node slot index 13, permutation: [0 2 1]
>>> Pre-location of node slot 13, RMSD: 2.92E-02
>>> Use given permutation for node slot index 14, permutation: [1 0 2]
>>> Pre-location of node slot 14, RMSD: 2.92E-02
>>> Use given permutation for node slot index 15, permutation: [0 2 1]
>>> Pre-location of node slot 15, RMSD: 2.92E-02
>>> Use given permutation for node slot index 16, permutation: [2 1 0]
>>> Pre-location of node slot 16, RMSD: 2.92E-02
>>> Use given permutation for node slot index 17, permutation: [0 1 2]
>>> Pre-location of node slot 17, RMSD: 2.92E-02
>>> Use given permutation for node slot index 18, permutation: [1 2 0]
>>> Pre-location of node slot 18, RMSD: 2.92E-02
>>> Use given permutation for node slot index 19, permutation: [2 0 1]
>>> Pre-location of node slot 19, RMSD: 2.92E-02
>>> Use given permutation for node slot index 20, permutation: [0 2 1]
>>> Pre-location of node slot 20, RMSD: 2.92E-02
>>> Use given permutation for node slot 

zyg


>>> == Min RMSD of (node type: 2, node bb: PcM_O): 4.38E-02
>>> Pre-location at node slot 16, (node type: 2, node bb: PcM_O), RMSD: 4.38E-02
>>> Pre-location at node slot 17, (node type: 2, node bb: PcM_O), RMSD: 9.78E-01
>>> RMSD > MIN_RMSD*1.01, relocate Node 17 with 216 trial orientations, RMSD: 4.38E-02
>>> Pre-location at node slot 18, (node type: 2, node bb: PcM_O), RMSD: 4.38E-02
>>> Pre-location at node slot 19, (node type: 2, node bb: PcM_O), RMSD: 4.38E-02
>>> Pre-location at node slot 20, (node type: 2, node bb: PcM_O), RMSD: 9.78E-01
>>> RMSD > MIN_RMSD*1.01, relocate Node 20 with 216 trial orientations, RMSD: 4.38E-02
>>> Pre-location at node slot 21, (node type: 2, node bb: PcM_O), RMSD: 4.38E-02
>>> Use given permutation for node slot index 0, permutation: [2 1 0]
>>> Pre-location of node slot 0, RMSD: 3.88E-07
>>> Use given permutation for node slot index 1, permutation: [0 2 1]
>>> Pre-location of node slot 1, RMSD: 3.88E-07
>>> Use given permutation for node slot inde

In [6]:
successful_cases

['bpc',
 'bpi',
 'cdl',
 'cdm',
 'cdn',
 'cds',
 'cds-b',
 'cdz',
 'clh',
 'etc',
 'hof',
 'mot',
 'mou',
 'nbo',
 'nta',
 'ssd',
 'sse',
 'ssf',
 'sst',
 'tfc',
 'tfe',
 'tfn',
 'tfz',
 'ths',
 'twt',
 'utp']